In [2]:
from wordlist import WordList
from learner import MyLearner

In [3]:
a = WordList()

In [ ]:
a.add_word_input()

In [ ]:
a.empty_the_list()


In [ ]:
b = a.search_word('de auto')
b

In [4]:
c = a.get_all_words()
c

[{'id': 1,
  'word': 'fel',
  'translation': 'bright',
  'translation2': 'яркий',
  'score': 0.0},
 {'id': 2,
  'word': 'telkens',
  'translation': 'each time, every time, again and again',
  'translation2': 'каждый раз, снова и снова',
  'score': 0.0},
 {'id': 3,
  'word': 'het suizen',
  'translation': 'whistling',
  'translation2': 'свист, шум, шипение',
  'score': 0.0},
 {'id': 4,
  'word': 'het geruis',
  'translation': 'rustling',
  'translation2': 'шуршание, шелест',
  'score': 0.0},
 {'id': 5,
  'word': 'vergissen',
  'translation': 'to be mistaken',
  'translation2': 'ошибаться',
  'score': 0.0},
 {'id': 6,
  'word': 'het zuiden',
  'translation': 'south',
  'translation2': 'юг',
  'score': 0.0},
 {'id': 7,
  'word': 'bewijzen',
  'translation': 'to prove',
  'translation2': 'доказывать',
  'score': 0.0},
 {'id': 8,
  'word': 'de nagel',
  'translation': 'nail',
  'translation2': 'ноготь',
  'score': 0.0},
 {'id': 9,
  'word': 'het tikken',
  'translation': 'ticking',
  'trans

In [ ]:
from IPython.display import Markdown

# Beautiful printing of the text
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(chat_completion.candidates[0].content.parts[0].text)

In [5]:
# prepare the llm model
import glob
import os
import re
import pathlib
import time
import textwrap
import google.generativeai as genai

# read the api_key from a text file
with open("var.txt","r") as f:
    var = f.read()
# connect to gemini model    
genai.configure(api_key=var)
model = genai.GenerativeModel('models/gemini-2.0-flash')

c:\Users\smith\Documents\python_scripts\quizgpt\learn_dutch\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'var.txt'

In [ ]:
# read ingestion list from a text file
with open('input3.txt', 'r') as f:
    data = f.read()


In [ ]:
# get a verified list from gemini
prompt_text = """
Here is a list of dutch words and short phrases. Some of them are translated to english and russian, some only to english. 
Add missing translations of dutch words to english or to russian columns and return it in json format. Make sure that there 
are no missing or null translations in english or in russian. The json format should be the following:
{'althans': {'english': 'at least', 'russian': 'по крайней мере'},
'overleggen': {'english': 'to discuss', 'russian': 'обсуждать'},
'het hoorcollege': {'english': 'lecture', 'russian': 'лекция'},
}
The words: """ 
chat_completion = model.generate_content(prompt_text + data)



In [ ]:
# correct the list and convert it to json and then to 2d list

import json
raw_string = chat_completion.candidates[0].content.parts[0].text

# 1. Clean up the Markdown code block wrappers
clean_json = raw_string.strip().removeprefix('```json').removesuffix('```').strip()

# 2. Parse it with json.loads()
data = json.loads(clean_json)

# 3. Convert it to a 2D lis
words_list = []
for dutch_word in data:
    dutch = dutch_word
    try:
      english = data[dutch_word]['english']
    except KeyError:
       english = ''
    try: 
      russian = data[dutch_word]['russian']
    except KeyError:
       russian = ''
    words_list.append([dutch, english, russian]) 



In [ ]:
# ingest the list to the list
a.ingest_list(words_list=words_list)

In [ ]:
from difflib import SequenceMatcher

def get_similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

similarity = get_similarity("het geweld", "de geweld")  # maybe 0.66
similarity

In [35]:
from wordlist import WordList
import random
import numpy as np
class MyLearner:
    def __init__(self, words_list:WordList):
        self.status = 0
        self.words_list = words_list
        self.words_items = self.words_list.get_all_words()
        self.n_list = len(self.words_list.get_all_words())
        self.threshold0 = 0.3
        self.threshold1 = 0.7
        # Portion of words in each group
        self.partition = [0.6, 0.3, 0.1]
    def SelectWords(self,n:int=10):
        # n, int, number of words to test
        score_list = [item['score'] for item in self.words_items]
        sorted_indices = np.argsort(score_list)
        print("Sorted indices:", sorted_indices)  # [1, 3, 0, 2]
        ind0 = [x for x in score_list if x < self.threshold0]
        ind1 = [x for x in score_list if (x >= self.threshold0 and x < self.threshold1)]
        ind2 = [x for x in score_list if x >= self.threshold1]
        ind_len = [len(ind0), len(ind1), len(ind2)]
        if self.n_list >= n:
            n0 = round(n*self.partition[0])
            n1 = round(n*self.partition[1])
            n2 = n - (n1 + n0)
            print(n0,n1,n2)
            if ind_len[0] >= n0:
                scored0 = random.sample(ind0,n0)
            else:
                scored0 = random.sample(ind0,ind_len[0])
            if ind_len[1] >= n1:
                scored1 = random.sample(ind1,n1)
            else:
                scored1 = random.sample(ind1,ind_len[1])
            if ind_len[2] >= n2:
                scored2 = random.sample(ind2,n2)
            else:
                scored2 = random.sample(ind2,ind_len[2])
        else:
            print(f"""Error: Requested too many words (n={n}). 
                  List contains only {self.n_list} words.""")
        # print(score_list,sorted_indices )




In [37]:
# from learner import MyLearner
d = MyLearner(a)
d.SelectWords(n=10)  


Sorted indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
6 3 1


In [29]:
import random

q = [random.random() for _ in range(10)]
b = random.sample(q,10)
sum(q)

4.6669563727313825